In [ ]:
from indra import literature 
import numpy as np
import pymysql
from Bio import Entrez
import itertools as it
import time
import networkx as nx
import pandas as pd
from bs4 import BeautifulSoup, Comment
import requests
from prettytable import PrettyTable
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import matplotlib.pyplot as plt
%matplotlib inline
# import ssl

# try:
#     _create_unverified_https_context = ssl._create_unverified_context
# except AttributeError:
#     pass
# else:
#     ssl._create_default_https_context = _create_unverified_https_context

# nltk.download()
# from nltk.tokenize import sent_tokenize
# nltk.download('punkt')
# nltk.download()

In [ ]:
"""
function return dict gene Entrez 2 Symbols
"""
def genent2sym():

    db = pymysql.connect("menchelabdb.int.cemm.at","readonly","ra4Roh7ohdee","GenesGO")    

    # prepare a cursor object using cursor() method
    cursor = db.cursor()

    sql = """   SELECT
                    Approved_Symbol,
                    Entrez_Gene_ID_NCBI 
                FROM GenesGO.hgnc_complete
                WHERE Entrez_Gene_ID_NCBI != ''
          """ 

    cursor.execute(sql)
    data = cursor.fetchall()    
#     try: 
#         # execute SQL query using execute() method.
#         cursor.execute(sql)
#         data = cursor.fetchall()
#     except:
#         print('SQL error')
    db.close()

#     t0 = time.time()
    d_sym_ent = {}
    d_ent_sym = {}

    for x in data:
        sym = x[0]
        ent = x[1]
        d_sym_ent[sym] = ent
        d_ent_sym[ent] = sym
#     print(time.time()-t0)
    
    return d_ent_sym, d_sym_ent


"""
function return dict pub_id 2 titles
"""
def genent2pubtitle(entrez):

    db = pymysql.connect("menchelabdb.int.cemm.at","readonly","ra4Roh7ohdee","GenesGO")    

    # prepare a cursor object using cursor() method
    cursor = db.cursor()

    sql = """   SELECT
                    tx.pubid,
                    tx.title
                FROM Datadivr.gene2pubid gp
                INNER JOIN Datadivr.pubid2text tx
                ON gp.pubid = tx.pubid
                WHERE gp.entrez = '%s'
          """ %entrez

    cursor.execute(sql)
    data = cursor.fetchall()    
#     try: 
#         # execute SQL query using execute() method.
#         cursor.execute(sql)
#         data = cursor.fetchall()
#     except:
#         print('SQL error')
    db.close()

#     t0 = time.time()
    d_pid_tlt = {}

    for x in data:
        d_pid_tlt[x[0]] = x[1]
    
    return d_pid_tlt


"""
function return title+abstacrt 
"""
def pubid2authorabstract(pubid):

    db = pymysql.connect("menchelabdb.int.cemm.at","readonly","ra4Roh7ohdee","GenesGO")    

    # prepare a cursor object using cursor() method
    cursor = db.cursor()

    sql = """   SELECT
                    author,
                    abstract
                FROM Datadivr.pubid2text
                WHERE pubid = '%s'
          """ %pubid

    cursor.execute(sql)
    data = cursor.fetchall()    
#     try: 
#         # execute SQL query using execute() method.
#         cursor.execute(sql)
#         data = cursor.fetchall()
#     except:
#         print('SQL error')
    db.close()

#     t0 = time.time()
#     l_text = []

    for x in data:
        text_str = x[0] + '<br>' + x[1]
#         l_text.append(text_str)
    
    return text_str


"""
function gets search string return l_entrez
"""
def genesfromstr(searchstr):

    searchstr_mod = '%'+searchstr+'%' 
    db = pymysql.connect("menchelabdb.int.cemm.at","readonly","ra4Roh7ohdee","GenesGO")    

    # prepare a cursor object using cursor() method
    cursor = db.cursor()

    sql = """   SELECT
                DISTINCT
                gp.entrez
                #pubid
                FROM Datadivr.pubid2text tx 
                INNER JOIN Datadivr.gene2pubid gp
                ON gp.pubid = tx.pubid
                WHERE tx.title LIKE '%s'
          """ %searchstr_mod

    cursor.execute(sql)
    data = cursor.fetchall()    
#     try: 
#         # execute SQL query using execute() method.
#         cursor.execute(sql)
#         data = cursor.fetchall()
#     except:
#         print('SQL error')
    db.close()

#     t0 = time.time()
    l_entz = []

    for x in data:
#         text_str = x[0] + '<br>' + x[1]
        l_entz.append(x[0])
    
    return l_entz



"""
function gets title and abstract as text body for gene
"""
def textfromgene(gene_symbol):

    db = pymysql.connect("menchelabdb.int.cemm.at","readonly","ra4Roh7ohdee","GenesGO")    

    # prepare a cursor object using cursor() method
    cursor = db.cursor()

    sql = """   SELECT
                    t.pubid,
                    t.title,
                    t.abstract
                FROM Datadivr.gene2pubid g
                INNER JOIN Datadivr.pubid2text t
                ON g.pubid = t.pubid
                WHERE g.symbol = '%s'
          """ %gene_symbol

    cursor.execute(sql)
    data = cursor.fetchall()    
#     try: 
#         # execute SQL query using execute() method.
#         cursor.execute(sql)
#         data = cursor.fetchall()
#     except:
#         print('SQL error')
    db.close()

#     t0 = time.time()
    d_pid_text = {}

    for x in data:
        text_str = x[1] + x[2]
#         l_text.append(text_str)
        d_pid_text[x[0]] = text_str
    return d_pid_text



"""
function returns list of all pubids from database
"""
def getallpids():

    db = pymysql.connect("asimov.westeurope.cloudapp.azure.com","readonly","ra4Roh7ohdee","Datadivr")    

    # prepare a cursor object using cursor() method
    cursor = db.cursor()

    sql = """   SELECT DISTINCT
                    t.pubid
                FROM Datadivr.gene2pubid t

          """ 

    cursor.execute(sql)
    data = cursor.fetchall()    
#     try: 
#         # execute SQL query using execute() method.
#         cursor.execute(sql)
#         data = cursor.fetchall()
#     except:
#         print('SQL error')
    db.close()

#     t0 = time.time()
    l_pid = []

    for x in data:
        l_pid.append(x[0])
    return l_pid


# GET DATE

In [ ]:
f = open('pidate.csv','w')

#generate list of pids
l_pids = getallpids()
print(len(l_pids))
t0 = time.time()
Entrez.email = "fmueller@cemm.oeaw.ac.at"

d_pid_date = {}
for pid in l_pids[:100]:
    handle = Entrez.esummary(db="pubmed", id=pid)
    record = Entrez.read(handle)
    handle.close()
    f.write('%s|%s\n' %(pid,record[0]["PubDate"]))
#     d_pid_date[pid] = record[0]["PubDate"]
f.close()

print('run time: %s' %(time.time()-t0))

In [ ]:
d_pid_daterk = {}
rank = 1
for a,b in sorted(d_pid_date.items(), key = lambda x: x[1]):
    print(a,b)
    d_pid_daterk[a] = rank
    rank += 1

# POST PROCESS PUBID 2 DATE FILE

In [58]:
d_month_number = {'Jan': '01','Feb': '02','Mar': '03','Apr': '04','May': '05','Jun': '06','Jul': '07','Aug': '08',
                  'Sep': '09','Oct': '10','Nov': '11','Dec': '12',
                 'January': '01','February': '02','March': '03','April': '04','June': '06','July': '07','August': '08',
                  'September': '09','October': '10','November': '11','Decemver': '12'}

f_db = open('Data/pidate_db.csv','w')

f = open('Data/pidate.csv','r')
lines = f.readlines()
f.close()



c = 0
for line in lines:#[c:]:
    
#     if line.find('|1')<0:
#         print(c)
#     print(line.find('|1')<0)
    
#     if line.find('|1') < 0 or line.find('|2') < 0:
#         print(c)
#############################################    
#     print(c)
    pid = line.split('|')[0]
    date_list = line.split('|')[1].split(' ')#[0].strip()
#     print(date_list)
    if len(date_list) == 1:
        year = date_list[0].strip() 
        if year.find('-') != -1:
            year = year.split('-')[0]        
        date_str = year + '-01-01'
    elif len(date_list) == 2:
        year = date_list[0].strip() 
        month = date_list[1].strip() 
        if month.find('-') != -1:
            month = month.split('-')[0]
#             print(c)
        date_str = year + '-' + d_month_number[month] + '-01'
        
    elif len(date_list) == 3:
        year = date_list[0].strip() 
        month = date_list[1].strip() 
        if month.find('-') != -1:
            month = month.split('-')[0]
        
        day = date_list[2].strip()
        if day.find('-'): 
            day = day.split('-')[0]
        if len(day) == 1:
            day = '0' + day
    
        date_str = year + '-' + d_month_number[month] + '-' + day

    f_db.write('%s|%s\n' %(pid,date_str))

#############################################    
#     print(c,pid,date_str)
    c += 1
f_db.close()